In [1]:
import numpy as np

# Generate synthetic data for clients
def generate_data(num_clients, num_samples, num_features):
    X = np.random.randn(num_clients, num_samples, num_features)
    y = np.random.randn(num_clients, num_samples)
    return X, y

# Epsilon-greedy neighbor sampling
def epsilon_greedy_selection(weights, epsilon, num_neighbors):
    """ Selects neighbors using epsilon-greedy strategy based on the collaboration weights. """
    neighbors = []
    for i in range(len(weights)):
        if np.random.rand() < epsilon:
            neighbors.append(np.random.choice(len(weights)))  # Random neighbor
        else:
            neighbors.append(np.argmax(weights))  # Best neighbor based on weights
    return np.random.choice(neighbors, num_neighbors, replace=False)

# E-step: Compute the posterior probability of each client choosing others as collaborators
def e_step(X, y, w, clients_models, num_clients, neighbors, beta, prev_losses, prev_weighted_losses):
    updated_losses = np.copy(prev_losses)
    updated_weighted_losses = np.copy(prev_weighted_losses)
    
    for i in range(num_clients):
        for neighbor in neighbors[i]:
            # Update loss for sampled neighbors
            y_pred = X[i] @ clients_models[neighbor]
            loss = np.mean((y[i] - y_pred) ** 2)
            updated_losses[i][neighbor] = loss
        
        # Exponential moving average for loss
        updated_weighted_losses[i] = (1 - beta) * prev_weighted_losses[i] + beta * updated_losses[i]

        # Update weights using softmax
        w[i] = np.exp(-updated_weighted_losses[i]) / np.sum(np.exp(-updated_weighted_losses[i]))
    
    return w, updated_losses, updated_weighted_losses

# M-step: Update models based on weighted collaborations
def m_step(X, y, w, clients_models, num_clients, neighbors, learning_rate):
    for i in range(num_clients):
        total_grad = np.zeros(clients_models[i].shape)
        
        for neighbor in neighbors[i]:
            grad = X[i].T @ (X[i] @ clients_models[neighbor] - y[i])  # Gradient of loss function
            total_grad += w[i][neighbor] * grad
        
        # Update the model using gradient descent
        clients_models[i] -= learning_rate * total_grad / len(y[i])
    
    return clients_models

# Main FedeRiCo algorithm
def federico(X, y, num_clients, num_features, num_rounds=100, num_neighbors=3, epsilon=0.1, beta=0.9, learning_rate=0.01):
    clients_models = np.random.randn(num_clients, num_features)  # Initialize models
    w = np.ones((num_clients, num_clients)) / num_clients  # Initialize uniform weights
    prev_losses = np.zeros((num_clients, num_clients))  # Initialize previous losses
    prev_weighted_losses = np.zeros((num_clients, num_clients))  # Initialize exponential moving avg losses

    for t in range(num_rounds):
        for client in num_clients:
            # Epsilon-greedy selection of neighbors
            neighbors = epsilon_greedy_selection(w[client], epsilon, num_neighbors)
            
            # E-step: Update the collaboration weights and losses
#             w, prev_losses, prev_weighted_losses = e_step(X, y, w, clients_models, num_clients, neighbors, beta, prev_losses, prev_weighted_losses)
            updated_weighted_losses = np.copy(prev_weighted_losses)

            for i in range(num_clients):
                for neighbor in neighbors[i]:
                    # Update loss for sampled neighbors
                    y_pred = X[i] @ clients_models[neighbor]
                    loss = np.mean((y[i] - y_pred) ** 2)
                    prev_losses[i][neighbor] = loss

                # Exponential moving average for loss
                updated_weighted_losses[i] = (1 - beta) * prev_weighted_losses[i] + beta * prev_losses[i]

                # Update weights using softmax
                w[i] = np.exp(-updated_weighted_losses[i]) / np.sum(np.exp(-updated_weighted_losses[i]))
            prev_weighted_losses = updated_weighted_losses
    
            
            # M-step: Update models based on collaboration weights
#             clients_models = m_step(X, y, w, clients_models, num_clients, neighbors, learning_rate)

            for i in range(num_clients):
                total_grad = np.zeros(clients_models[i].shape)

                for neighbor in neighbors[i]:
                    grad = X[i].T @ (X[i] @ clients_models[neighbor] - y[i])  # Gradient of loss function
                    total_grad += w[i][neighbor] * grad

                # Update the model using gradient descent
                clients_models[i] -= learning_rate * total_grad / len(y[i])
            
            
#         # Epsilon-greedy selection of neighbors
#         neighbors = [epsilon_greedy_selection(w[i], epsilon, num_neighbors) for i in range(num_clients)]
        
#         # E-step: Update the collaboration weights and losses
#         w, prev_losses, prev_weighted_losses = e_step(X, y, w, clients_models, num_clients, neighbors, beta, prev_losses, prev_weighted_losses)
        
#         # M-step: Update models based on collaboration weights
#         clients_models = m_step(X, y, w, clients_models, num_clients, neighbors, learning_rate)
    
    return clients_models, w

# Example usage
num_clients = 5
num_samples = 100
num_features = 10
X, y = generate_data(num_clients, num_samples, num_features)

# Run FedeRiCo algorithm
clients_models, weights = federico(X, y, num_clients, num_features)
print("Final client models:\n", clients_models)
print("Final collaboration weights:\n", weights)

Final client models:
 [[ 0.19926457 -0.21069184 -0.01492785 -0.04499039 -0.0567861   0.01299461
   0.22351153 -0.18349235  0.11014492  0.03991851]
 [-1.49562347 -0.37298621  1.4313522   0.08702984  1.87530007 -2.14766631
   0.05317328  1.66340646 -0.48522196  1.38526575]
 [-0.2537337   1.78952453  1.51502062  0.50210929  0.70595557 -3.62362947
   0.45946401  0.55037266 -1.87310847 -0.03795609]
 [-1.20177641  0.53839458  1.49391117  1.35011958 -0.36996364  0.56356795
  -3.43042271 -0.0519068  -1.76844198 -0.80624311]
 [-1.7879493  -0.6909482   1.70989277  1.01397328  0.10405149 -0.5016159
  -0.40071077 -0.34341689  0.17289195 -0.78262408]]
Final collaboration weights:
 [[9.99841938e-01 3.54264781e-10 3.95815059e-09 2.02757486e-10
  1.58057133e-04]
 [9.99214984e-01 1.27167072e-06 1.37106071e-11 3.42859130e-09
  7.83740886e-04]
 [9.99891696e-01 4.46096144e-07 8.81346185e-13 5.17584705e-12
  1.07858194e-04]
 [9.99834657e-01 3.51379763e-08 5.20572580e-09 8.45147040e-10
  1.65301899e-04]
 [9